In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from tqdm import tqdm  # Import tqdm for progress bar
from fake_useragent import UserAgent  # <-- Thêm thư viện giả lập User-Agent
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import re
import ast  # để chuyển từ string thành list thật

#---------
from urllib.request import urlopen, Request

In [2]:
# Đọc nội dung file txt
with open('all_post_url.txt', 'r', encoding='utf-8') as f:
    content = f.read()

# Chuyển từ chuỗi sang list thực
url_list = ast.literal_eval(content)

# Tạo DataFrame từ list
df_url = pd.DataFrame(url_list, columns=['url'])

print(df_url.shape)  # In ra (số_url, 1)
df_url.head()  # Xem vài dòng đầu

(4235, 1)


,url
0,https://www.topcv.vn/viec-lam/korean-bridge-so...
1,https://www.topcv.vn/viec-lam/network-technica...
2,https://www.topcv.vn/viec-lam/gaming-sop-desig...
3,https://www.topcv.vn/viec-lam/lap-trinh-vien-b...
4,https://www.topcv.vn/viec-lam/chuyen-vien-kiem...


In [3]:
import subprocess
import time
import socket

def check_port_open(port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('localhost', port))
    sock.close()
    return result == 0  # Port is open if result is 0

# Khởi động Chrome với remote-debugging port
subprocess.Popen([
    r'C:\Program Files\Google\Chrome\Application\chrome.exe',
    '--remote-debugging-port=8797',
    r'--user-data-dir=C:\Users\Administrator\Downloads\chromedriver-win64\chromedriver-win64\localhost'  # Profile riêng
])

# Đợi đến khi Chrome sẵn sàng
while not check_port_open(8797):
    time.sleep(0.5)

## 🧠 1. Setup driver (nối với Chrome đang mở)

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def connect_to_driver(debugger_address="localhost:8797"):
    options = Options()
    options.add_experimental_option("debuggerAddress", debugger_address)
    driver = webdriver.Chrome(options=options)
    return driver

## 🌐 2. Hàm crawl 1 URL

In [5]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from datetime import datetime
import time
import random

def crawl_url(driver, url, timeout=10):
    try:
        start_time = datetime.now()
        driver.get(url)

        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        html = driver.page_source
        return html, start_time, None
    except Exception as e:
        print(f"❌ Lỗi khi xử lý URL: {url} - {e}")
        return None, None, url

## 🔁 3. Hàm crawl nhiều URL

In [6]:
from tqdm import tqdm

def crawl_urls(driver, df, url_column='url', sleep_range=(10, 20)):
    html_contents = []
    fail_urls = []
    timestamps = []

    for url in tqdm(df[url_column]):
        html, timestamp, fail_url = crawl_url(driver, url)
        html_contents.append(html)
        timestamps.append(timestamp)
        if fail_url:
            fail_urls.append(fail_url)

        time.sleep(random.randint(*sleep_range))

    df = df.copy()
    df['html'] = html_contents
    df['crawl_start_time'] = timestamps
    return df, fail_urls

## 🔁 4. Hàm lọc các URL bị Cloudflare chặn (html lang="en-US")

In [7]:
def filter_cloudflare_block(df, html_column='html'):
    return df[df[html_column].str.contains('<html lang="en-US" dir', na=False)]

## 💥 Sử dụng hàm dễ dàng như chơi game:

In [8]:
# 1. Kết nối tới trình duyệt
driver = connect_to_driver()

# 2. Lấy sample URL
df_sample = df_url[3000:4000].copy()

# 3. Thêm cột retry_count
df_sample['retry_count'] = 0

# 4. Cào lần đầu
df_sample, fail_urls = crawl_urls(driver, df_sample)

# 5. Lưu tạm kết quả vào file (để load lại khi retry)
from tqdm import tqdm

with open('pass_0.json', 'w', encoding='utf-8') as f:
    for _, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
        f.write(row.to_json(force_ascii=False) + '\n')

 16%|█▌        | 157/1000 [55:18<4:54:12, 20.94s/it]

❌ Lỗi khi xử lý URL: https://www.topcv.vn/viec-lam/lap-trinh-vien-unity-mobile-game/1145456.html?ta_source=ITJobs_LinkDetail - HTTPConnectionPool(host='localhost', port=50438): Read timed out. (read timeout=120)


 16%|█▌        | 158/1000 [57:37<13:14:05, 56.59s/it]

❌ Lỗi khi xử lý URL: https://www.topcv.vn/viec-lam/ky-su-speech-processing-engineer/1686879.html?ta_source=ITJobs_LinkDetail - HTTPConnectionPool(host='localhost', port=50438): Read timed out. (read timeout=120)


 16%|█▌        | 159/1000 [59:54<18:51:21, 80.72s/it]

❌ Lỗi khi xử lý URL: https://www.topcv.vn/viec-lam/automation-tester/1686880.html?ta_source=ITJobs_LinkDetail - HTTPConnectionPool(host='localhost', port=50438): Read timed out. (read timeout=120)


 16%|█▌        | 160/1000 [1:02:09<22:38:03, 97.00s/it]

❌ Lỗi khi xử lý URL: https://www.topcv.vn/viec-lam/ios-developer/1686873.html?ta_source=ITJobs_LinkDetail - HTTPConnectionPool(host='localhost', port=50438): Read timed out. (read timeout=120)


100%|██████████| 1000/1000 [00:38<00:00, 26.15it/s]


In [8]:
import os

# === 1. Kết nối lại trình duyệt (nếu cần) ===
driver = connect_to_driver()

# === 2. Tìm file gần nhất để tiếp tục ===
# Lấy danh sách file pass dạng: pass_0.json, pass_1.json, ...
pass_files = sorted([f for f in os.listdir() if f.startswith("pass_") and f.endswith(".json")])

if not pass_files:
    raise FileNotFoundError("❌ Không tìm thấy file crawl đầu tiên (pass_0.json). Hãy chạy cell crawl trước.")

# Lấy file mới nhất để tiếp tục retry
last_pass = pass_files[-1]
rows = []
with open(last_pass, 'r', encoding='utf-8') as f:
    for line in f:
        row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
        rows.append(row)

df_sample = pd.DataFrame(rows)

# === 3. Retry Cloudflare ===
max_retries = 1  # mỗi lần chỉ chạy 1 vòng để kiểm soát bằng tay
retry_count = 0

while retry_count < max_retries:
    df_blocked = filter_cloudflare_block(df_sample)

    if df_blocked.empty:
        print("🎉 Không còn URL nào bị chặn. Lưu kết quả vào final.json")
        df_sample.to_json("final.json", orient='records', indent=2, force_ascii=False)
        break

    print(f"⚡ Retry từ {last_pass} → Lần retry thứ {retry_count + 1}: {len(df_blocked)} URL bị chặn...")

    # Tăng số lần retry
    df_sample.loc[df_blocked.index, 'retry_count'] += 1

    # Crawl lại
    df_retry, retry_fails = crawl_urls(driver, df_blocked)

    # Cập nhật lại HTML mới vào sample
    df_sample.update(df_retry)

    # Tăng số bước pass
    pass_index = int(last_pass.split("_")[1].split(".")[0]) + 1
    new_pass_filename = f"pass_{pass_index}.json"

    # Ghi ra file mới cho lần retry này
    with open(new_pass_filename, 'w', encoding='utf-8') as f:
        for _, row in df_sample.iterrows():
            f.write(row.to_json(force_ascii=False) + '\n')
        print(f"✅ Đã lưu phiên bản mới: {new_pass_filename}")

    retry_count += 1

C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_8012\658445289.py:18: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_8012\658445289.py:18: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_8012\658445289.py:18: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
C:\Users\Administrator\AppData\

⚡ Retry từ pass_0.json → Lần retry thứ 1: 387 URL bị chặn...


100%|██████████| 387/387 [2:11:35<00:00, 20.40s/it]  
C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_8012\658445289.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1746257603802.0 1746257622567.0 1746257639192.0 1746257656343.0
 1746257673040.0 1746257691939.0 1746257715412.0 1746257732906.0
 1746257748535.0 1746257772054.0 1746257789500.0 1746257806832.0
 1746257825658.0 1746257849600.0 1746257874571.0 1746257894497.0
 1746257914814.0 1746257936141.0 1746257953871.0 1746257972313.0
 1746257992547.0 1746258013268.0 1746258039782.0 1746258067028.0
 1746258094146.0 1746258118741.0 1746258151375.0 1746258172851.0
 1746258197749.0 1746258216150.0 1746258236334.0 1746258258126.0
 1746258284272.0 1746258311090.0 1746258337024.0 1746258356014.0
 1746258374619.0 1746258395132.0 1746258419909.0 1746258439545.0
 1746258458375.0 1746258480640.0 1746258503927.0 1746258527958.0
 1746258546632.0 1746258564828.0 1746

✅ Đã lưu phiên bản mới: pass_1.json


In [8]:
import os

# === 1. Kết nối lại trình duyệt (nếu cần) ===
driver = connect_to_driver()

# === 2. Tìm file gần nhất để tiếp tục ===
# Lấy danh sách file pass dạng: pass_0.json, pass_1.json, ...
pass_files = sorted([f for f in os.listdir() if f.startswith("pass_") and f.endswith(".json")])

if not pass_files:
    raise FileNotFoundError("❌ Không tìm thấy file crawl đầu tiên (pass_0.json). Hãy chạy cell crawl trước.")

# Lấy file mới nhất để tiếp tục retry
last_pass = pass_files[-1]
rows = []
with open(last_pass, 'r', encoding='utf-8') as f:
    for line in f:
        row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
        rows.append(row)

df_sample = pd.DataFrame(rows)

# === 3. Retry Cloudflare ===
max_retries = 1  # mỗi lần chỉ chạy 1 vòng để kiểm soát bằng tay
retry_count = 0

while retry_count < max_retries:
    df_blocked = filter_cloudflare_block(df_sample)

    if df_blocked.empty:
        print("🎉 Không còn URL nào bị chặn. Lưu kết quả vào final.json")
        df_sample.to_json("final.json", orient='records', indent=2, force_ascii=False)
        break

    print(f"⚡ Retry từ {last_pass} → Lần retry thứ {retry_count + 1}: {len(df_blocked)} URL bị chặn...")

    # Tăng số lần retry
    df_sample.loc[df_blocked.index, 'retry_count'] += 1

    # Crawl lại
    df_retry, retry_fails = crawl_urls(driver, df_blocked)

    # Cập nhật lại HTML mới vào sample
    df_sample.update(df_retry)

    # Tăng số bước pass
    pass_index = int(last_pass.split("_")[1].split(".")[0]) + 1
    new_pass_filename = f"pass_{pass_index}.json"

    # Ghi ra file mới cho lần retry này
    with open(new_pass_filename, 'w', encoding='utf-8') as f:
        for _, row in df_sample.iterrows():
            f.write(row.to_json(force_ascii=False) + '\n')
        print(f"✅ Đã lưu phiên bản mới: {new_pass_filename}")

    retry_count += 1

C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_6112\658445289.py:18: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_6112\658445289.py:18: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_6112\658445289.py:18: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
C:\Users\Administrator\AppData\

⚡ Retry từ pass_1.json → Lần retry thứ 1: 8 URL bị chặn...


100%|██████████| 8/8 [03:32<00:00, 26.50s/it]
C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_6112\658445289.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1746209504526 1746209525381 1746209544830 1746209568334 1746209593520
 1746209613471 1746209632030 1746209647323 1746209663961 1746209688887
 1746209706179 1746209728030 1746209746963 1746209760903 1746209775375
 1746209789188 1746209814976 1746209830816 1746209857152 1746209872926
 1746209897127 1746209917944 1746209940751 1746209963689 1746209990010
 1746210013795 1746210036736 1746210059372 1746210079629 1746210097569
 1746210123235 1746210138516 1746210162586 1746210186852 1746210213021
 1746210234722 1746210254713 1746210276214 1746210292724 1746210312625
 1746210330589 1746210353032 1746210368488 1746210394612 1746210421365
 1746210439497 1746210457321 1746210474846 1746210495133 1746210513657
 1746210534409 1746210550368 1746210568360 1746210

✅ Đã lưu phiên bản mới: pass_2.json


In [8]:
import os

# === 1. Kết nối lại trình duyệt (nếu cần) ===
driver = connect_to_driver()

# === 2. Tìm file gần nhất để tiếp tục ===
# Lấy danh sách file pass dạng: pass_0.json, pass_1.json, ...
pass_files = sorted([f for f in os.listdir() if f.startswith("pass_") and f.endswith(".json")])

if not pass_files:
    raise FileNotFoundError("❌ Không tìm thấy file crawl đầu tiên (pass_0.json). Hãy chạy cell crawl trước.")

# Lấy file mới nhất để tiếp tục retry
last_pass = pass_files[-1]
rows = []
with open(last_pass, 'r', encoding='utf-8') as f:
    for line in f:
        row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
        rows.append(row)

df_sample = pd.DataFrame(rows)

# === 3. Retry Cloudflare ===
max_retries = 1  # mỗi lần chỉ chạy 1 vòng để kiểm soát bằng tay
retry_count = 0

while retry_count < max_retries:
    df_blocked = filter_cloudflare_block(df_sample)

    if df_blocked.empty:
        print("🎉 Không còn URL nào bị chặn. Lưu kết quả vào final.json")
        df_sample.to_json("final.json", orient='records', indent=2, force_ascii=False)
        break

    print(f"⚡ Retry từ {last_pass} → Lần retry thứ {retry_count + 1}: {len(df_blocked)} URL bị chặn...")

    # Tăng số lần retry
    df_sample.loc[df_blocked.index, 'retry_count'] += 1

    # Crawl lại
    df_retry, retry_fails = crawl_urls(driver, df_blocked)

    # Cập nhật lại HTML mới vào sample
    df_sample.update(df_retry)

    # Tăng số bước pass
    pass_index = int(last_pass.split("_")[1].split(".")[0]) + 1
    new_pass_filename = f"pass_{pass_index}.json"

    # Ghi ra file mới cho lần retry này
    with open(new_pass_filename, 'w', encoding='utf-8') as f:
        for _, row in df_sample.iterrows():
            f.write(row.to_json(force_ascii=False) + '\n')
        print(f"✅ Đã lưu phiên bản mới: {new_pass_filename}")

    retry_count += 1

C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_6592\658445289.py:18: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_6592\658445289.py:18: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_6592\658445289.py:18: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
C:\Users\Administrator\AppData\

⚡ Retry từ pass_1.json → Lần retry thứ 1: 46 URL bị chặn...


100%|██████████| 46/46 [16:05<00:00, 21.00s/it]
C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_6592\658445289.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1746257603802.0 1746257622567.0 1746257639192.0 1746257656343.0
 1746257673040.0 1746257691939.0 1746257715412.0 1746257732906.0
 1746257748535.0 1746257772054.0 1746257789500.0 1746257806832.0
 1746257825658.0 1746257849600.0 1746257874571.0 1746257894497.0
 1746257914814.0 1746257936141.0 1746257953871.0 1746257972313.0
 1746257992547.0 1746258013268.0 1746258039782.0 1746258067028.0
 1746258094146.0 1746258118741.0 1746258151375.0 1746258172851.0
 1746258197749.0 1746258216150.0 1746258236334.0 1746258258126.0
 1746258284272.0 1746258311090.0 1746258337024.0 1746258356014.0
 1746258374619.0 1746258395132.0 1746258419909.0 1746258439545.0
 1746258458375.0 1746258480640.0 1746258503927.0 1746258527958.0
 1746258546632.0 1746258564828.0 1746258581

✅ Đã lưu phiên bản mới: pass_2.json


In [8]:
import os

# === 1. Kết nối lại trình duyệt (nếu cần) ===
driver = connect_to_driver()

# === 2. Tìm file gần nhất để tiếp tục ===
# Lấy danh sách file pass dạng: pass_0.json, pass_1.json, ...
pass_files = sorted([f for f in os.listdir() if f.startswith("pass_") and f.endswith(".json")])

if not pass_files:
    raise FileNotFoundError("❌ Không tìm thấy file crawl đầu tiên (pass_0.json). Hãy chạy cell crawl trước.")

# Lấy file mới nhất để tiếp tục retry
last_pass = pass_files[-1]
rows = []
with open(last_pass, 'r', encoding='utf-8') as f:
    for line in f:
        row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
        rows.append(row)

df_sample = pd.DataFrame(rows)

# === 3. Retry Cloudflare ===
max_retries = 1  # mỗi lần chỉ chạy 1 vòng để kiểm soát bằng tay
retry_count = 0

while retry_count < max_retries:
    df_blocked = filter_cloudflare_block(df_sample)

    if df_blocked.empty:
        print("🎉 Không còn URL nào bị chặn. Lưu kết quả vào final.json")
        
        with open("final.json", 'w', encoding='utf-8') as f:
            for _, row in df_sample.iterrows():
                f.write(row.to_json(force_ascii=False) + '\n')
            print(f"✅ Đã lưu phiên bản final.json")
        break

    print(f"⚡ Retry từ {last_pass} → Lần retry thứ {retry_count + 1}: {len(df_blocked)} URL bị chặn...")

    # Tăng số lần retry
    df_sample.loc[df_blocked.index, 'retry_count'] += 1

    # Crawl lại
    df_retry, retry_fails = crawl_urls(driver, df_blocked)

    # Cập nhật lại HTML mới vào sample
    df_sample.update(df_retry)

    # Tăng số bước pass
    pass_index = int(last_pass.split("_")[1].split(".")[0]) + 1
    new_pass_filename = f"pass_{pass_index}.json"

    # Ghi ra file mới cho lần retry này
    with open(new_pass_filename, 'w', encoding='utf-8') as f:
        for _, row in df_sample.iterrows():
            f.write(row.to_json(force_ascii=False) + '\n')
        print(f"✅ Đã lưu phiên bản mới: {new_pass_filename}")

    retry_count += 1

C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_9028\3645135668.py:18: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_9028\3645135668.py:18: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_9028\3645135668.py:18: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  row = pd.read_json(line, typ='series')  # hoặc: pd.Series(json.loads(line))
C:\Users\Administrator\AppDa

🎉 Không còn URL nào bị chặn. Lưu kết quả vào final.json
✅ Đã lưu phiên bản final.json
